In [1]:
from selenium import webdriver
import selenium
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
driver = webdriver.Chrome()
driver.set_page_load_timeout(15)
driver.get('https://sellercentral.amazon.com/forums/c/account-health')

for i in range(0,3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

data = driver.find_elements_by_css_selector('a[class = "title raw-link raw-topic-link"]')
titles = [x.text for x in data]
links = [x.get_attribute('href') for x in data]

data_subcat = driver.find_elements_by_css_selector('span[class = "category-name"]')
sub_cats = [x.text for x in data_subcat[2:]]

contents = []
for link in links:
    driver.get(link)
    data = driver.find_elements_by_css_selector('div[class = "cooked"]')
    content_tempo = [x.text for x in data]
    contents.append(content_tempo)

time.sleep(5)
driver.quit()

In [3]:
df_responses = pd.DataFrame(contents)
df_responses.insert(loc = 0, column = 'links', value = links)

In [5]:
init_content = [i[0] for i in contents]
df = pd.DataFrame(titles, columns=['titles'])
df['category'] = 'Account Health'
df['sub category'] = sub_cats
df['links'] = links
df['contents'] = init_content

In [6]:
df.head()

,titles,category,sub category,links,contents
0,How to change main image while suspended,Account Health,Account Health,https://sellercentral.amazon.com/forums/t/how-...,"Hi there,\nI’m currently suspended for a false..."
1,Policy warning for “Adding duplicate child var...,Account Health,Account Health,https://sellercentral.amazon.com/forums/t/poli...,After creating a new parent-child relationship...
2,Suspended Account,Account Health,Account Health,https://sellercentral.amazon.com/forums/t/susp...,"Hi there,\nI have uploaded every documents whi..."
3,Account suspended because it’s linked to anoth...,Account Health,Account Health,https://sellercentral.amazon.com/forums/t/acco...,I just recently signed up for a sellers accoun...
4,Account deactivated without valid reason and n...,Account Health,Related Accounts,https://sellercentral.amazon.com/forums/t/acco...,Account was deactivated with following reason;...


In [7]:
df_responses.head()

,links,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,https://sellercentral.amazon.com/forums/t/how-...,"Hi there,\nI’m currently suspended for a false...",I’m pretty sure you can’t.\nIt is my understan...,"Hi @HaveaGoodTime,\nSophie from Amazon here.\n...",Thank you Sophie. They tried by that method an...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,https://sellercentral.amazon.com/forums/t/poli...,After creating a new parent-child relationship...,If you have more than one FBA listing for New ...,Thank you! We have a large number of what I ca...,"Hello @jayfiveseller,\nCassie with Amazon here...",@Cassie_Amazon thank you so much for the offic...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,https://sellercentral.amazon.com/forums/t/susp...,"Hi there,\nI have uploaded every documents whi...",It can take Amazon 30+ days to respond.,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,https://sellercentral.amazon.com/forums/t/acco...,I just recently signed up for a sellers accoun...,You will receive no clarification of what acco...,Thank you so much for responding. I’m assuming...,It is not a linking to your buyer account that...,I have never had another seller account or kno...,"Interview your relatives, friends, business as...",So If I can’t determine what this supposed lin...,Did you use your computer to sign up?\nDid you...,Thanks for your reply. I used my computer (des...,...,No I’ve never had another sellers account. I s...,"They linked you somehow, sorry, I don’t know w...",thanks for your help,Wanderlust_Products:\nThe email I have associa...,"Hello @Wanderlust_Products,\nCecilia here.\nI ...",Hi Cecilia.\nI am appreciative of the advise t...,What concerns Amazon and what it seeks to prev...,"Will the OP’s chance for a second attempt, eve...",michael_mma:\nWill the OP’s chance for a secon...,None
4,https://sellercentral.amazon.com/forums/t/acco...,Account was deactivated with following reason;...,We are having the same exact issue. You have b...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
